In [1]:
import polars as pl
df = pl.read_csv("../data/data.csv")

In [ ]:
print(df.columns)
print(df.dtypes)

# Intervalos anómalos

In [3]:
import datetime 

anomalies = [
    (datetime.datetime(2020, 4, 12, 11, 50), datetime.datetime(2020, 4, 12, 23, 30)),
    (datetime.datetime(2020, 4, 18, 0, 0), datetime.datetime(2020, 4, 18, 23, 59)),
    (datetime.datetime(2020, 4, 19, 0, 0), datetime.datetime(2020, 4, 19, 1, 30)),
    (datetime.datetime(2020, 4, 29, 3, 20), datetime.datetime(2020, 4, 29, 4, 0)),
    (datetime.datetime(2020, 4, 29, 22, 0), datetime.datetime(2020, 4, 29, 22, 20)),
    (datetime.datetime(2020, 5, 13, 14, 0), datetime.datetime(2020, 5, 13, 23, 59)),
    (datetime.datetime(2020, 5, 18, 5, 0), datetime.datetime(2020, 5, 18, 5, 30)),
    (datetime.datetime(2020, 5, 19, 10, 10), datetime.datetime(2020, 5, 19, 11, 0)),
    (datetime.datetime(2020, 5, 19, 22, 10), datetime.datetime(2020, 5, 19, 23, 59)),
    (datetime.datetime(2020, 5, 20, 0, 0), datetime.datetime(2020, 5, 20, 20, 0)),
    (datetime.datetime(2020, 5, 23, 9, 50), datetime.datetime(2020, 5, 23, 10, 10)),
    (datetime.datetime(2020, 5, 29, 23, 30), datetime.datetime(2020, 5, 29, 23, 59)),
    (datetime.datetime(2020, 5, 30, 0, 0), datetime.datetime(2020, 5, 30, 6, 0)),
    (datetime.datetime(2020, 6, 1, 15, 0), datetime.datetime(2020, 6, 1, 15, 40)),
    (datetime.datetime(2020, 6, 3, 10, 0), datetime.datetime(2020, 6, 3, 11, 0)),
    (datetime.datetime(2020, 6, 5, 10, 0), datetime.datetime(2020, 6, 5, 23, 59)),
    (datetime.datetime(2020, 6, 6, 0, 0), datetime.datetime(2020, 6, 6, 23, 59)),
    (datetime.datetime(2020, 6, 7, 0, 0), datetime.datetime(2020, 6, 7, 14, 30)),
    (datetime.datetime(2020, 7, 8, 17, 30), datetime.datetime(2020, 7, 8, 19, 0)),
    (datetime.datetime(2020, 7, 15, 14, 30), datetime.datetime(2020, 7, 15, 19, 0)),
    (datetime.datetime(2020, 7, 17, 4, 30), datetime.datetime(2020, 7, 17, 5, 30)),
]


raros = [
    #NO APARECE COMO ANOMALIA
    (datetime.datetime(2020, 3, 6, 21, 42, 15), datetime.datetime(2020, 3, 6, 23, 14, 0)),
    (datetime.datetime(2020, 3, 11, 5, 15, 10), datetime.datetime(2020, 3, 11, 6, 25, 0)),
    (datetime.datetime(2020, 3, 12, 0, 15, 56), datetime.datetime(2020, 3, 12, 11, 59, 0)),
    (datetime.datetime(2020, 3, 26, 4, 0, 20), datetime.datetime(2020, 3, 26, 5, 20, 0)),
    (datetime.datetime(2020, 3, 27, 7, 12, 0), datetime.datetime(2020, 3, 27, 12, 1, 0)),
    (datetime.datetime(2020, 4, 17, 8, 50, 28), datetime.datetime(2020, 4, 17, 23, 59, 0)),
    (datetime.datetime(2020, 4, 25, 0, 7, 15), datetime.datetime(2020, 4, 25, 1, 10, 0)),
    (datetime.datetime(2020, 5, 19, 1, 35, 28), datetime.datetime(2020, 5, 19, 2, 40, 0)),
    (datetime.datetime(2020, 6, 12, 1, 41, 7), datetime.datetime(2020, 6, 12, 17, 6, 0)),
    (datetime.datetime(2020, 7, 21, 13, 32, 48), datetime.datetime(2020, 7, 21, 22, 3, 0)),
    (datetime.datetime(2020, 7, 22, 6, 40, 46), datetime.datetime(2020, 7, 22, 13, 10, 0)),
    (datetime.datetime(2020, 7, 31, 0, 57, 33), datetime.datetime(2020, 7, 31, 2, 9, 0))
]

anomalies.extend(raros)

# Correción de periodos temporales.
final_anomalies = []
for anomaly in anomalies: 
  new = list(anomaly)
  if anomaly[1].minute == 59:
    new[1] += datetime.timedelta(minutes=1)
  final_anomalies.append(new)

In [4]:
import datetime
df = df.with_columns(pl.col("timestamp").str.to_datetime())
df = df.with_row_index("rownr")

In [ ]:
anomalies[1][1] + datetime.timedelta(seconds=59)

# Definimos las anomalias

In [6]:
def is_anomaly(instance_date, anomalies: list[datetime]):
    flag_anomaly = False
    index = 0
    while not flag_anomaly and index < len(anomalies):
        t = anomalies[index]
        if instance_date >= t[0] and instance_date <= t[1] + datetime.timedelta(seconds=59):
            flag_anomaly = True
        index += 1
    return flag_anomaly


df = df.select(pl.all(), pl.lit(0).alias("is_anomaly"))
for anomaly in final_anomalies:
  df = df.with_columns((pl.col("is_anomaly") + df["timestamp"].is_between(anomaly[0], anomaly[1])))
df = df.select(pl.exclude("is_anomaly"), pl.col("is_anomaly") >= 1)
# df = df.select(pl.all(), pl.col("timestamp").map_elements(lambda x: is_anomaly(x, anomalies), return_dtype=pl.Boolean).alias("is_anomaly"))

# Justificamos el cálculo de la ventana

In [ ]:
# Create transition states
tdf = df.select(pl.all(), pl.col("is_anomaly").shift(-1).alias("next_is_anomaly"), pl.col("Motor_current").gt(0.05).alias("motor_state"))
tdf = tdf.select(pl.all(), (pl.col("is_anomaly") != pl.col("next_is_anomaly")).alias("transition"))
tdf

In [ ]:
# Create groups based on transition
tdf = tdf.select(pl.all(), pl.col("transition").cum_sum().alias("groups"))
tdf["groups"].max()

In [ ]:
gdf = tdf.select(pl.all(), pl.col("groups").shift(-1).alias("next_group"), pl.col("motor_state").shift(-1).alias("next_motor_state"))
# Filter a ON switch
fdf = gdf.filter((pl.col("motor_state") == 0) & (pl.col("next_motor_state") == 1)).with_columns(pl.col("timestamp").shift(-1).alias("next_timestamp"))
# Filter out different time groups
fdf = fdf.filter(pl.col("groups") == pl.col("next_group"))
# Calculate the duration
fdf = fdf.with_columns((pl.col("next_timestamp") - pl.col("timestamp")).alias("duration")).filter(pl.col("duration").is_not_null())
print(fdf["duration"].dt.total_seconds().max(), fdf["duration"].dt.total_seconds().min(), fdf["duration"].dt.total_seconds().mean(), fdf["duration"].dt.total_seconds().mode()[0])

In [ ]:
fdf.filter(pl.col("duration").eq(pl.col("duration").max()))

In [ ]:
import plotly.express as px
data_anomaly = df.filter(pl.col("timestamp").is_between(
datetime.datetime(2020,6,5,00,00,00), datetime.datetime(2020,6,8,15,00,00)
))
px.line(data_anomaly, x="timestamp", y=["TP3", "Motor_current"])

In [ ]:
data_anomaly.select(pl.col("is_anomaly"), pl.col("is_anomaly").shift(-1).alias("next_is_anomaly"), pl.col("timestamp"), pl.col("timestamp").shift(-1).alias("next_timestamp")).filter(~pl.col("is_anomaly").eq(pl.col("next_is_anomaly")))

## Para cada grupo

In [ ]:
gdf = tdf.select(pl.all(), pl.col("groups").shift(-1).alias("next_group"), pl.col("motor_state").shift(-1).alias("next_motor_state"))
# Filter a ON switch
durations = []
for label, group in gdf.filter(pl.col("groups") == pl.col("next_group")).group_by("groups"):
  tmp = group.filter((pl.col("motor_state") == 0) & (pl.col("next_motor_state") == 1)).with_columns(pl.col("timestamp").shift(-1).alias("next_timestamp"))
  tmp = tmp.with_columns((pl.col("next_timestamp") - pl.col("timestamp")).alias("duration")).filter(pl.col("duration").is_not_null())
  durations += tmp["duration"].to_list()
durations = [x.total_seconds() for x in durations]
print(max(durations), min(durations), sum(durations) / len(durations), durations[len(durations)//2])

## Para el primer tramo no-anómalo

In [ ]:
subdf = df.filter(pl.col("timestamp").lt(anomalies[0][0]))
subdf = subdf.with_columns(pl.col("Motor_current").lt(0.05).alias("motor_state"))
subdf.with_columns(pl.col("motor_state").shift(-1).alias("next_motor_state")).filter((pl.col("motor_state") == 0) & (pl.col("next_motor_state") == 1))["timestamp"].diff().dt.total_seconds().mode()[0]

## Para cada grupo evitando saltos temporales

In [ ]:
durations = []
subdf = df.filter(pl.col("timestamp").lt(final_anomalies[0][0]))
subdf = subdf.with_columns(pl.col("Motor_current").lt(0.05).alias("motor_state"))
subdf = subdf.with_columns(pl.col("motor_state").shift(-1).alias("next_motor_state")).filter((pl.col("motor_state") == 0) & (pl.col("next_motor_state") == 1))
subdf = subdf.with_columns(pl.col("timestamp").diff().dt.total_seconds().alias("duration")).filter(pl.col("duration").is_not_null())
print(len(durations), len(subdf))
durations += subdf["duration"].to_list()
for anomaly, next_anomaly, next_next_anomaly in zip(final_anomalies, final_anomalies[1:], final_anomalies[2:]):
  subdf = df.filter(
    (pl.col("timestamp").lt(next_anomaly[0]) & (pl.col("timestamp").gt(anomaly[1])))
  )
  subdf = subdf.with_columns(pl.col("Motor_current").lt(0.05).alias("motor_state"))
  subdf = subdf.with_columns(pl.col("motor_state").shift(-1).alias("next_motor_state")).filter((pl.col("motor_state") == 0) & (pl.col("next_motor_state") == 1))
  subdf = subdf.with_columns(pl.col("timestamp").diff().dt.total_seconds().alias("duration")).filter(pl.col("duration").is_not_null())
  print(len(durations), len(subdf))
  durations += subdf["duration"].to_list()
  subdf = df.filter(
    (pl.col("timestamp").gt(next_anomaly[1]) & (pl.col("timestamp").lt(next_next_anomaly[0])))
  )
  subdf = subdf.with_columns(pl.col("Motor_current").lt(0.05).alias("motor_state"))
  subdf = subdf.with_columns(pl.col("motor_state").shift(-1).alias("next_motor_state")).filter((pl.col("motor_state") == 0) & (pl.col("next_motor_state") == 1))
  subdf = subdf.with_columns(pl.col("timestamp").diff().dt.total_seconds().alias("duration")).filter(pl.col("duration").is_not_null())
  print(len(durations), len(subdf))
  durations += subdf["duration"].to_list()
print(max(durations), min(durations), sum(durations) / len(durations), sorted(durations)[len(durations)//2])

In [16]:
df = df.with_columns(pl.col("timestamp").diff().cum_sum().alias("diff"))

In [17]:
df = df.with_columns(pl.col("timestamp").diff().cum_sum().alias("diff"))
df = df.with_columns((pl.col("diff").dt.total_seconds() // (1249 * 2)).alias("id"))
df = df.with_columns(pl.coalesce(pl.col("id").cast(pl.UInt64), 0).alias("id"))

In [ ]:
df.group_by("id", maintain_order=True).len()

In [ ]:
import plotly.express as px
import pandas as pd
start = 51000
end = 51000 + 10000
px.line(df[start : end], x="timestamp", y="TP3", color="id")

# Intentamos rellenar valores pérdidos. 

In [ ]:
import plotly.express as px
import pandas as pd
start = 51000
end = 51000 + 10000
test = pd.read_csv("../data/data.csv")
test = test.assign(TP3L = test.TP3.interpolate(method="linear"))
px.line(test[start : end], x="timestamp", y="TP3L")

Tendríamos que imputar solamente aquellos tramos cuya distancia fuera casi insignificante (algunos minutos). 
Ya que si observamos el Feb-9T00:00 a Feb-9T06:00 hemos introducido una línea similar a lo que seŕia una anomalía por el tamaño del salto temporal.

Otra opción sería investigar introducir segmentos cíclicos hasta que la diferencia fuera manejable. No obstante, esto sería difícil de argumentar y confuso para los posibles segmentos anómalos. 

## Observar valores faltantes

In [ ]:
with pl.Config(tbl_rows=1000):
  print(df.filter(pl.col("timestamp").gt(datetime.datetime(2020,2, 1, 19, 40, 00)) & pl.col("timestamp").lt(datetime.datetime(2020, 2, 2, 6, 00, 00))))

In [ ]:
df.filter(pl.col("timestamp").diff().dt.total_seconds().gt(10))

# Verificar valores raros

In [ ]:
px.line(df.filter(pl.col("timestamp").is_between(*raros[5])), x="timestamp", y="TP3", color="id")

# Revisar datos procesados

In [ ]:
import polars as pl 

df = pl.read_csv("../data/data.csv")
df

In [ ]:
tiempo_medio_ciclo = 1260
df.filter(pl.col("timestamp").str.to_datetime().diff().dt.total_seconds() > tiempo_medio_ciclo * 2)

# Creación del conjunto de datos

In [ ]:
import polars as pl
tiempo_medio_ciclo = 1260
df = pl.read_csv("../data/data.csv")
# Str timestamp to datetime object
df = df.select(pl.exclude("timestamp"), pl.col("timestamp").str.to_datetime())
# Calculate distance from next read
df = df.with_columns(pl.col("timestamp").diff().cum_sum().alias("diff"))
# Divide by mean cicle
df = df.with_columns((pl.col("diff").dt.total_seconds() // (tiempo_medio_ciclo * 2)).alias("id"))
# Count how many different samples are in each sliding sample 
df = df.join(df.group_by("id").len("count"), on="id", how="inner")
# Filter those windows that have less than half points
df = df.filter(pl.col("count") >= (tiempo_medio_ciclo)/ (10))
# Cast the ID column 
df = df.with_columns(pl.col("id").cast(pl.UInt64).alias("id"))
# Asignamos la etiqueta
df = df.select(pl.all(), pl.lit(0).alias("is_anomaly"))
for anomaly in final_anomalies:
  df = df.with_columns((pl.col("is_anomaly") + df["timestamp"].is_between(anomaly[0], anomaly[1])))
df = df.select(pl.exclude("is_anomaly"), pl.col("is_anomaly") >= 1)
# Visualize a sample and the group length
df.group_by("id").first().with_columns(pl.col("count").min().alias("smallest_window"))

In [ ]:
# Agrupamos las variables que vamos a utilizar.
a = ['TP2',
     'TP3',
     'H1',
     'DV_pressure',
     'Reservoirs',
     'Oil_temperature',
     'Motor_current',
     'COMP',
     'DV_eletric',
     'Towers',
     'MPG',
     'LPS',
     'Pressure_switch',
     'Oil_level',
     'Caudal_impulses',
]

# Ahora obtenemos los diferentes ids que vamos a reducir (ventanas)
df_X = df.select("id").group_by("id", maintain_order=True).first()

# Calculamos las características
for c in a:
  df_aux = df.group_by("id", maintain_order=True).agg([
      pl.col(f"{c}").mean().alias(f"{c}_mean"),
      pl.col(f"{c}").max().alias(f"{c}_max"),
      pl.col(f"{c}").min().alias(f"{c}_min"),
      pl.col(f"{c}").median().alias(f"{c}_median"),
      pl.col(f"{c}").var().alias(f"{c}_var"),
  ])

  df_X = df_X.join(df_aux, on="id", how="left")
df_X

In [ ]:
# Etiquetamos según mayoría en la ventana.
labels = df.select(["id", "is_anomaly"]).group_by(pl.all()).len("count")
labels = labels.select(pl.all(), pl.col("count").max().over("id").alias("democracy"))
labels = labels.filter(pl.col("count").eq(pl.col("democracy"))).select(["id", "is_anomaly"])

# Etiquetamos las anomalías
df_X = df_X.join(labels, how="inner", on="id")
df_X = df_X.drop("id").with_row_index("id")
df_X

In [29]:
seed = 140421
#Separamos anomalias de no anomalias
df_no_anomaly = df_X.filter(pl.col("is_anomaly") == False)
df_anomaly = df_X.filter(pl.col("is_anomaly") == True)

#Agrupamos las que pertenecen a una misma anomalia
df_anomaly = df_anomaly.sort("id").with_columns(pl.col("id").shift(1).fill_null(0).alias("last_id"))

df_anomaly = df_anomaly.with_columns(((pl.col("id") - pl.col("last_id")) >= 2).alias("group"))

df_anomaly = df_anomaly.with_columns(pl.col("group").cum_sum())
df_anomaly = df_anomaly.join(df_anomaly.select("id", "group").group_by("group", maintain_order=True).len(name="count"),
                             on="group", how="inner")

#Creamos grupos de los grupos con un mismo tamaño para un k prefijado
k = 9
test_anomaly_size = int(df_anomaly.shape[0] / k)
df_groups = df_anomaly.group_by("group", maintain_order=True).first().select("group", "count")
df_groups = df_groups.sample(fraction=1, seed=seed, shuffle=True)

groups = []
for i in df_groups.to_dicts():
    index = 0
    flag_assigned = False
    while index < len(groups) and not flag_assigned:
        if groups[index]["count"] + i["count"] < test_anomaly_size:
            groups[index]["group"].append(i["group"])
            groups[index]["count"] += i["count"]
            flag_assigned = True
            index = 0
        else:
            index += 1
    if not flag_assigned and len(groups) >= index:
        if len(groups) < k:
            groups.append({"count": i["count"], "group": [i["group"]]})
        else:
            min_size = min([a["count"] for a in groups])
            for index in range(0, k):
                if groups[index]["count"] == min_size:
                    groups[index]["group"].append(i["group"])
                    groups[index]["count"] += i["count"]
                    flag_assigned = True
                    index = 0

In [30]:
seed = 140421
#Repetimos el proceso para las no anomalias
df_no_anomaly = df_no_anomaly.sort("id").with_columns(pl.col("id").shift(1).fill_null(0).alias("last_id"))

df_no_anomaly = df_no_anomaly.with_columns(((pl.col("id") - pl.col("last_id")) >= 2).alias("group"))

df_no_anomaly = df_no_anomaly.with_columns(pl.col("group").cum_sum())
df_no_anomaly = df_no_anomaly.join(df_no_anomaly.select("id", "group").group_by("group").len(name="count"),
                                   on="group", how="inner")

#Creamos grupos de los grupos con un mismo tamaño para un k prefijado
test_anomaly_size = int(df_no_anomaly.shape[0] / k)
df_groups_no_anomaly = df_no_anomaly.group_by("group", maintain_order=True).first().select("group", "count")
df_groups_no_anomaly = df_groups_no_anomaly.sample(fraction=1, seed=seed, shuffle=True)

groups_no_anomaly = []
for i in df_groups_no_anomaly.to_dicts():
    index = 0
    flag_assigned = False
    while index < len(groups_no_anomaly) and not flag_assigned:
        if groups_no_anomaly[index]["count"] + i["count"] < test_anomaly_size:
            groups_no_anomaly[index]["group"].append(i["group"])
            groups_no_anomaly[index]["count"] += i["count"]
            flag_assigned = True
            index = 0
        else:
            index += 1
    if not flag_assigned and len(groups_no_anomaly) >= index:
        if len(groups_no_anomaly) < k:
            groups_no_anomaly.append({"count": i["count"], "group": [i["group"]]})
        else:
            min_size = min([a["count"] for a in groups_no_anomaly])
            for index in range(0, k):
                if groups_no_anomaly[index]["count"] == min_size:
                    groups_no_anomaly[index]["group"].append(i["group"])
                    groups_no_anomaly[index]["count"] += i["count"]
                    flag_assigned = True
                    index = 0


In [31]:
split = 0
df_X = df_X.with_columns(pl.lit(None).alias("split_where_test"))
for a, b in zip(groups, groups_no_anomaly):
    split_ids = df_anomaly.filter(pl.col("group").is_in(a["group"]))["id"].to_list() + \
                df_no_anomaly.filter(pl.col("group").is_in(b["group"]))["id"].to_list()
    df_X = df_X.with_columns(
        pl.when((pl.col("id").is_in(split_ids))).then(split).otherwise(pl.col("split_where_test")).alias("split_where_test"))
    split += 1

In [ ]:
df_X.group_by("split_where_test", "is_anomaly").len(name="count").sort("split_where_test", "is_anomaly")

In [33]:
df_test = df_X.filter(pl.col("split_where_test").is_in([1,8]))
df_test = df_test.select(pl.exclude(["id","split_where_test"]))


df_training = df_X.filter(~pl.col("split_where_test").is_in([1,8]))
df_training = df_training.with_columns(
                                       pl.when(pl.col("split_where_test").is_in([0,7]))
                                       .then(0)
                                       .when(pl.col("split_where_test").is_in([2,6]))
                                       .then(1)
                                       .when(pl.col("split_where_test").is_in([5,3]))
                                       .then(2)
                                       .when(pl.col("split_where_test").is_in([4]))
                                       .then(3).alias("split_where_test_2"))
df_training = df_training.select(pl.exclude(["id","split_where_test"]))
df_training = df_training.rename({"split_where_test_2": "fold"})

In [ ]:
df_training.group_by("fold", "is_anomaly").len(name="count").sort("fold", "is_anomaly")

In [ ]:
index = 0
for a in groups:
    if index not in [1,8,4]:
        print("index: ",index ,"count: ",a.get("count"))
    index += 1 


In [36]:
df_training.write_csv("../data/train.csv")
df_test.write_csv("../data/test.csv")

# Entrenar modelo Naive Bayes

In [ ]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
import numpy as np

np.random.seed(140421)

# Load the training and testing data
train_data = pd.read_csv('../data/train.csv')
train_data.reset_index()
test_data = pd.read_csv('../data/test.csv')
test_data.reset_index()

target_column = 'is_anomaly'
fold_column = 'fold'
feature_columns = [col for col in train_data.columns if col not in [target_column, fold_column]]

X_train = train_data[feature_columns]
y_train = train_data[target_column]

def get_cv_iterable(
  folds: list,
  fold_column: str,
  train: pd.DataFrame,
): 
  for fold in folds:
    test_indexes = train[train[fold_column] == fold].index
    train_indexes = train[train[fold_column] != fold].index
    yield (train_indexes, test_indexes)

X_test = test_data[feature_columns]
y_test = test_data[target_column]

# Set up cross-validation using the 'folds' column
folds = train_data[fold_column].unique()

cv = GridSearchCV(
  estimator = GaussianNB(), 
  param_grid={
    "var_smoothing": np.linspace(1e-11, 1e-7, 100)
  },
  cv = get_cv_iterable(folds, fold_column, train_data),
  scoring=[
    "f1",
    "recall",
    "precision", 
    "accuracy"
  ],
  refit="f1",
)

cv.fit(X_train, y_train)
with pd.option_context("display.max_columns", 33):
  display(pd.DataFrame(cv.cv_results_))
print(cv.best_estimator_, cv.best_score_)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

ypred_train = cv.best_estimator_.predict(X_train)
print(pd.DataFrame(classification_report(y_train, ypred_train, output_dict=True)))
_ = ConfusionMatrixDisplay.from_estimator(cv.best_estimator_, X_train, y_train)

# Instance and Feature Selection 

## CNN

In [ ]:
import pandas as pd
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import CondensedNearestNeighbour
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
import numpy as np

np.random.seed(140421)


# Load the training and testing data
train_data = pd.read_csv('../data/train.csv')
train_data.reset_index()
test_data = pd.read_csv('../data/test.csv')
test_data.reset_index()

target_column = 'is_anomaly'
fold_column = 'fold'
feature_columns = [col for col in train_data.columns if col not in [target_column, fold_column]]

X_train = train_data[feature_columns]
y_train = train_data[target_column]

def get_cv_iterable(
  folds: list,
  fold_column: str,
  train: pd.DataFrame,
): 
  for fold in folds:
    test_indexes = train[train[fold_column] == fold].index
    train_indexes = train[train[fold_column] != fold].index
    yield (train_indexes, test_indexes)

X_test = test_data[feature_columns]
y_test = test_data[target_column]

folds = train_data[fold_column].unique()
model = Pipeline([
        ('sampling', CondensedNearestNeighbour(random_state=seed)),
        ('nb', GaussianNB())
    ])

rcv = GridSearchCV(
  estimator = model, 
  param_grid={
    "nb_var_smoothing": np.linspace(1e-11, 1e-7, 100)
  },
  cv = get_cv_iterable(folds, fold_column, train_data),
  scoring=[
    "f1",
    "recall",
    "precision", 
    "accuracy"
  ],
  refit="f1",
)

rcv.fit(X_train, y_train)
with pd.option_context("display.max_columns", 33):
  display(pd.DataFrame(rcv.cv_results_))
print(rcv.best_estimator_, rcv.best_score_)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

ypred_train = rcv.best_estimator_.predict(X_train)
print(pd.DataFrame(classification_report(y_train, ypred_train, output_dict=True)))
_ = ConfusionMatrixDisplay.from_estimator(rcv.best_estimator_, X_train, y_train)

In [ ]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
import numpy as np

np.random.seed(140421)


# Load the training and testing data
train_data = pd.read_csv('../data/train.csv')
train_data.reset_index()
test_data = pd.read_csv('../data/test.csv')
test_data.reset_index()

target_column = 'is_anomaly'
fold_column = 'fold'
feature_columns = [col for col in train_data.columns if col not in [target_column, fold_column]]

X_train = train_data[feature_columns]
y_train = train_data[target_column]

def get_cv_iterable(
  folds: list,
  fold_column: str,
  train: pd.DataFrame,
): 
  for fold in folds:
    test_indexes = train[train[fold_column] == fold].index
    train_indexes = train[train[fold_column] != fold].index
    yield (train_indexes, test_indexes)

X_test = test_data[feature_columns]
y_test = test_data[target_column]

folds = train_data[fold_column].unique()
model = Pipeline([
        ('sampling', SMOTETomek(random_state=seed)),
        ('nb', GaussianNB())
    ])

rcv = GridSearchCV(
  estimator = model, 
  param_grid={
    "nb__var_smoothing": np.linspace(1e-11, 1e-7, 100)
  },
  cv = get_cv_iterable(folds, fold_column, train_data),
  scoring=[
    "f1",
    "recall",
    "precision", 
    "accuracy"
  ],
  refit="f1",
)

rcv.fit(X_train, y_train)
with pd.option_context("display.max_columns", 33):
  display(pd.DataFrame(rcv.cv_results_))
print(rcv.best_estimator_, rcv.best_score_)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

ypred_train = rcv.best_estimator_.predict(X_train)
print(pd.DataFrame(classification_report(y_train, ypred_train, output_dict=True)))
_ = ConfusionMatrixDisplay.from_estimator(rcv.best_estimator_, X_train, y_train)

## MRMR

In [ ]:
MRMR = [ 
  "Towers_mean", 
  "DV_eletric_var",
  "DV_pressure_median",
  "TP2_max",
  "Towers_median",        
  "Reservoirs_max",
  "DV_eletric_min",       
  "Pressure_switch_min", 
  "Motor_current_var",
  "Towers_var",
  "LPS_min",
  "DV_pressure_min",
  "Towers_min",
  "TP2_median", 
  "Oil_level_max",
  "Reservoirs_mean",
  "DV_pressure_mean",
  "Oil_temperature_min",
  "Oil_temperature_var",  
  "DV_eletric_median",
  "Caudal_impulses_max",
  "DV_pressure_var", 
  "LPS_max"              ,
  "H1_max",
  "Motor_current_max",
  "Motor_current_median",
  "COMP_min",
  # "TP2_var",
  # "DV_eletric_mean",      
  # "TP3_var",
  # "Reservoirs_median",
  # "H1_median",
]

# Load the training and testing data
train_data = pd.read_csv('../data/train.csv')
train_data.reset_index()
test_data = pd.read_csv('../data/test.csv')
test_data.reset_index()

target_column = 'is_anomaly'
fold_column = 'fold'
feature_columns = MRMR

X_train = train_data[feature_columns]
y_train = train_data[target_column]

def get_cv_iterable(
  folds: list,
  fold_column: str,
  train: pd.DataFrame,
): 
  for fold in folds:
    test_indexes = train[train[fold_column] == fold].index
    train_indexes = train[train[fold_column] != fold].index
    yield (train_indexes, test_indexes)

X_test = test_data[feature_columns]
y_test = test_data[target_column]

# Set up cross-validation using the 'folds' column
folds = train_data[fold_column].unique()

mcv = GridSearchCV(
  estimator = GaussianNB(), 
  param_grid={
    "var_smoothing": np.linspace(1e-11, 1e-7, 100)
  },
  cv = get_cv_iterable(folds, fold_column, train_data),
  scoring=[
    "f1",
    "recall",
    "precision", 
    "accuracy"
  ],
  refit="f1",
)

mcv.fit(X_train, y_train)
with pd.option_context("display.max_columns", 33):
  display(pd.DataFrame(mcv.cv_results_))
print(mcv.best_estimator_, mcv.best_score_)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

ypred_train = mcv.best_estimator_.predict(X_train)
print(pd.DataFrame(classification_report(y_train, ypred_train, output_dict=True)))
_ = ConfusionMatrixDisplay.from_estimator(mcv.best_estimator_, X_train, y_train)

## CHI-Squared

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2


# Load the training and testing data
train_data = pd.read_csv('../data/train.csv')
train_data.reset_index()
test_data = pd.read_csv('../data/test.csv')
test_data.reset_index()

target_column = 'is_anomaly'
fold_column = 'fold'
feature_columns = [col for col in train_data.columns if col not in [target_column, fold_column]]

X_train = train_data[feature_columns]
y_train = train_data[target_column]

def get_cv_iterable(
  folds: list,
  fold_column: str,
  train: pd.DataFrame,
): 
  for fold in folds:
    test_indexes = train[train[fold_column] == fold].index
    train_indexes = train[train[fold_column] != fold].index
    yield (train_indexes, test_indexes)

X_test = test_data[feature_columns]
y_test = test_data[target_column]

# Set up cross-validation using the 'folds' column
folds = train_data[fold_column].unique()
x_train = MinMaxScaler().fit_transform(X_train) 
chi_scores = chi2(x_train, y_train)
p_values = pd.Series(chi_scores[1],index = X_train.columns)
p_values.sort_values(ascending = False , inplace = True)
p_values.plot.bar()
plt.show()

indexes = chi_scores[1] <= 0.05
columns = X_train.columns
print(columns[indexes])

X_train = X_train.loc[:, indexes]

ccv = GridSearchCV(
  estimator = GaussianNB(), 
  param_grid={
    "var_smoothing": np.linspace(1e-11, 1e-7, 100)
  },
  cv = get_cv_iterable(folds, fold_column, train_data),
  scoring=[
    "f1",
    "recall",
    "precision", 
    "accuracy"
  ],
  refit="f1",
)

ccv.fit(X_train, y_train)
with pd.option_context("display.max_columns", 33):
  display(pd.DataFrame(ccv.cv_results_))
print(ccv.best_estimator_, ccv.best_score_)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

ypred_train = ccv.best_estimator_.predict(X_train)
print(pd.DataFrame(classification_report(y_train, ypred_train, output_dict=True)))
_ = ConfusionMatrixDisplay.from_estimator(ccv.best_estimator_, X_train, y_train)

## CNN + ChiSquared

In [ ]:
import matplotlib.pyplot as plt
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2


# Load the training and testing data
train_data = pd.read_csv('../data/train.csv')
train_data.reset_index()
test_data = pd.read_csv('../data/test.csv')
test_data.reset_index()

target_column = 'is_anomaly'
fold_column = 'fold'
# feature_columns = MRMR
feature_columns = [col for col in train_data.columns if col not in [target_column, fold_column]]

X_train = train_data[feature_columns]
y_train = train_data[target_column]

def get_cv_iterable(
  folds: list,
  fold_column: str,
  train: pd.DataFrame,
): 
  for fold in folds:
    test_indexes = train[train[fold_column] == fold].index
    train_indexes = train[train[fold_column] != fold].index
    yield (train_indexes, test_indexes)

X_test = test_data[feature_columns]
y_test = test_data[target_column]

# Set up cross-validation using the 'folds' column
folds = train_data[fold_column].unique()
x_train = MinMaxScaler().fit_transform(X_train) 
chi_scores = chi2(x_train, y_train)

indexes = chi_scores[1] <= 0.05
columns = X_train.columns
print(columns[indexes])


X_train = X_train.loc[:, indexes]

model = Pipeline([
        ('sampling', CondensedNearestNeighbour()),
        ('nb', GaussianNB())
    ])

frcv = GridSearchCV(
  estimator = model, 
  param_grid={
    "nb__var_smoothing": np.linspace(1e-11, 1e-7, 100)
  },
  cv = get_cv_iterable(folds, fold_column, train_data),
  scoring=[
    "f1",
    "recall",
    "precision", 
    "accuracy"
  ],
  refit="f1",
)

frcv.fit(X_train, y_train)
with pd.option_context("display.max_columns", 33):
  display(pd.DataFrame(frcv.cv_results_))
print(frcv.best_estimator_, frcv.best_score_)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

ypred_train = frcv.best_estimator_.predict(X_train)
print(pd.DataFrame(classification_report(y_train, ypred_train, output_dict=True)))
_ = ConfusionMatrixDisplay.from_estimator(frcv.best_estimator_, X_train, y_train)

# CNN + MRMR

In [ ]:
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.pipeline import Pipeline


# Load the training and testing data
train_data = pd.read_csv('../data/train.csv')
train_data.reset_index()
test_data = pd.read_csv('../data/test.csv')
test_data.reset_index()

target_column = 'is_anomaly'
fold_column = 'fold'
feature_columns = MRMR
# feature_columns = [col for col in train_data.columns if col not in [target_column, fold_column]]

X_train = train_data[feature_columns]
y_train = train_data[target_column]

def get_cv_iterable(
  folds: list,
  fold_column: str,
  train: pd.DataFrame,
): 
  for fold in folds:
    test_indexes = train[train[fold_column] == fold].index
    train_indexes = train[train[fold_column] != fold].index
    yield (train_indexes, test_indexes)

X_test = test_data[feature_columns]
y_test = test_data[target_column]

model = Pipeline([
        ('sampling', CondensedNearestNeighbour()),
        ('nb', GaussianNB())
    ])

frcv = GridSearchCV(
  estimator = model, 
  param_grid={
    "nb__var_smoothing": np.linspace(1e-11, 1e-7, 100)
  },
  cv = get_cv_iterable(folds, fold_column, train_data),
  scoring=[
    "f1",
    "recall",
    "precision", 
    "accuracy"
  ],
  refit="f1",
)

frcv.fit(X_train, y_train)
with pd.option_context("display.max_columns", 33):
  display(pd.DataFrame(frcv.cv_results_))
print(frcv.best_estimator_, frcv.best_score_)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

ypred_train = frcv.best_estimator_.predict(X_train[MRMR])
print(pd.DataFrame(classification_report(y_train, ypred_train, output_dict=True)))
_ = ConfusionMatrixDisplay.from_estimator(frcv.best_estimator_, X_train[MRMR], y_train)

## SMOTELinks + MRMR

In [ ]:
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline


# Load the training and testing data
train_data = pd.read_csv('../data/train.csv')
train_data.reset_index()
test_data = pd.read_csv('../data/test.csv')
test_data.reset_index()

target_column = 'is_anomaly'
fold_column = 'fold'
feature_columns = MRMR
# feature_columns = [col for col in train_data.columns if col not in [target_column, fold_column]]

X_train = train_data[feature_columns]
y_train = train_data[target_column]

def get_cv_iterable(
  folds: list,
  fold_column: str,
  train: pd.DataFrame,
): 
  for fold in folds:
    test_indexes = train[train[fold_column] == fold].index
    train_indexes = train[train[fold_column] != fold].index
    yield (train_indexes, test_indexes)

X_test = test_data[feature_columns]
y_test = test_data[target_column]

model = Pipeline([
        ('sampling', SMOTETomek(random_state=seed)),
        ('nb', GaussianNB())
    ])

frcv = GridSearchCV(
  estimator = model, 
  param_grid={
    "nb__var_smoothing": np.linspace(1e-11, 1e-7, 100)
  },
  cv = get_cv_iterable(folds, fold_column, train_data),
  scoring=[
    "f1",
    "recall",
    "precision", 
    "accuracy"
  ],
  refit="f1",
)

frcv.fit(X_train, y_train)
with pd.option_context("display.max_columns", 33):
  display(pd.DataFrame(frcv.cv_results_))
print(frcv.best_estimator_, frcv.best_score_)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

ypred_train = frcv.best_estimator_.predict(X_train[MRMR])
print(pd.DataFrame(classification_report(y_train, ypred_train, output_dict=True)))
_ = ConfusionMatrixDisplay.from_estimator(frcv.best_estimator_, X_train[MRMR], y_train)

# Entrenar el modelo de Stacking

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

seed = 140421
# Utilizamos una versión básica de stacking
estimators = [
    ('dt', DecisionTreeClassifier(random_state=seed)),
    ('svc', SVC(random_state=seed)),
    ('nb', GaussianNB())
]
# Con un clasificador final lineal
stack = StackingClassifier(
  estimators=estimators,
  final_estimator=LogisticRegression()
)
# Hacemos un GridSearch
scv = GridSearchCV(
  estimator = stack, 
  param_grid = {
    # Decision Tree 
    'dt__max_depth': [None, 10],  # Unlimited or moderate depth
    'dt__criterion': ['gini', 'entropy'],  # Gini or Entropy

    # SVC
    'svc__C': [0.1, 1, 10],  

    # Naive Bayes
    # 'nb__solver': [1e-9, 1e-8],  

    # Logistic Regression
    'final_estimator__penalty': [None, 'l2'], # Can focus on 1 model or be more moderate
  },
  cv = get_cv_iterable(folds, fold_column, train_data),
  scoring=[
    "f1",
    "recall",
    "precision", 
    "accuracy"
  ],
  refit="f1",
)

scv.fit(X_train, y_train)
with pd.option_context("display.max_columns", 33):
  display(pd.DataFrame(scv.cv_results_))
print(scv.best_estimator_, scv.best_score_)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

ypred_train = scv.best_estimator_.predict(X_train)
print(pd.DataFrame(classification_report(y_train, ypred_train, output_dict=True)))
_ = ConfusionMatrixDisplay.from_estimator(scv.best_estimator_, X_train, y_train)

In [ ]:
# Get the coefficients (assuming binary classification for simplicity)
coefficients = scv.best_estimator_.final_estimator_.coef_.flatten()  # Flatten if multi-dimensional
abs_coefficients = np.abs(coefficients)  # Get absolute values of coefficients
total_weight = np.sum(abs_coefficients)  # Sum of absolute weights

# Create a DataFrame for better visualization
data = {
    "Model Name": [name for name, _ in stack.estimators],
    "Absolute Weight": coefficients,
    "Relative Weight (%)": (abs_coefficients / total_weight) * 100
}
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

## Stacking + SMOTETomek

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline

seed = 140421
# Utilizamos una versión básica de stacking
estimators = [
    ('dt', DecisionTreeClassifier(random_state=seed)),
    ('svc', SVC(random_state=seed)),
    ('nb', GaussianNB())
]
# Con un clasificador final lineal
stack = StackingClassifier(
  estimators=estimators,
  final_estimator=LogisticRegression()
)

model = Pipeline([
        ('sampling', SMOTETomek(random_state=seed)),
        ('stack', stack)
    ])
# Hacemos un GridSearch
scv = GridSearchCV(
  estimator = model, 
  param_grid = {
    # Decision Tree 
    'stack__dt__max_depth': [None, 10],  # Unlimited or moderate depth
    'stack__dt__criterion': ['gini', 'entropy'],  # Gini or Entropy

    # SVC
    'stack__svc__C': [0.1, 1, 10],  

    # Naive Bayes
    # 'nb__solver': [1e-9, 1e-8],  

    # Logistic Regression
    'stack__final_estimator__penalty': [None, 'l2'], # Can focus on 1 model or be more moderate
  },
  cv = get_cv_iterable(folds, fold_column, train_data),
  scoring=[
    "f1",
    "recall",
    "precision", 
    "accuracy"
  ],
  refit="f1",
)

scv.fit(X_train, y_train)
with pd.option_context("display.max_columns", 33):
  display(pd.DataFrame(scv.cv_results_))
print(scv.best_estimator_, scv.best_score_)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

ypred_train = scv.best_estimator_.predict(X_train)
print(pd.DataFrame(classification_report(y_train, ypred_train, output_dict=True)))
_ = ConfusionMatrixDisplay.from_estimator(scv.best_estimator_, X_train, y_train)

In [ ]:
# Get the coefficients (assuming binary classification for simplicity)
coefficients = scv.best_estimator_.named_steps["stack"].final_estimator_.coef_.flatten() # Flatten if multi-dimensional
abs_coefficients = np.abs(coefficients)  # Get absolute values of coefficients
total_weight = np.sum(abs_coefficients)  # Sum of absolute weights

# Create a DataFrame for better visualization
data = {
    "Model Name": [name for name, _ in stack.estimators],
    "Absolute Weight": coefficients,
    "Relative Weight (%)": (abs_coefficients / total_weight) * 100
}
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

# Aplicaciones operaciones sobre test con los mejores modelos

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

ypred = cv.best_estimator_.predict(X_test)
print(pd.DataFrame(classification_report(y_test, ypred, output_dict=True)))
_ = ConfusionMatrixDisplay.from_estimator(cv.best_estimator_, X_test, y_test)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

ypred = scv.best_estimator_.predict(X_test)
print(pd.DataFrame(classification_report(y_test, ypred, output_dict=True)))
_ = ConfusionMatrixDisplay.from_estimator(scv.best_estimator_, X_test, y_test)